# Concordance Analysis: Positive SPECT/CT + Sestamibi Scans

### Objective

Assess **localization concordance with surgical findings** for patients with **positive SPECT/CT + Sestamibi scans**.

### Data Source

- `side_concordance_results.xlsx` (pre-operative imaging results and surgical outcomes)

### Outputs

- **Total scans with localization:** 285
- **Scans matching surgical side or exact gland:** 270 (**94.7%**)
---

In [2]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [3]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

In [4]:
def check_spect_sestamibi(row):
    spect = row['SPECT/CT']
    sestamibi = row['Sestamibi']
    
    spect_localizing = pd.notnull(spect) and spect not in ['No Scan', 'Non-localizing']
    sestamibi_localizing = pd.notnull(sestamibi) and sestamibi not in ['No Scan', 'Non-localizing']
    
    if not pd.notnull(spect) and not pd.notnull(sestamibi):
        return 'No SPECT/CT or Sestamibi Recorded'
    elif spect_localizing and sestamibi_localizing:
        return f"SPECT: {spect} | Sestamibi: {sestamibi}"
    elif spect_localizing:
        return f'SPECT/CT: {spect}'
    elif sestamibi_localizing:
        return f'Sestamibi: {sestamibi}'
    else:
        return 'Non-localizing'
    
preop_imaging_df['SPECT/CT-Sestamibi'] = preop_imaging_df.apply(check_spect_sestamibi, axis=1)
preop_imaging_df['SPECT/CT-Sestamibi'].head()

0    Sestamibi: Right Inferior
1               Non-localizing
2     Sestamibi: Left Inferior
3     Sestamibi: Left Inferior
4              Sestamibi: Left
Name: SPECT/CT-Sestamibi, dtype: object

In [5]:
def check_concordance(row, modality_col):
    imaging = row[modality_col]
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    # Exact gland match
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    # Side match
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'


,Procedure_Date,Patient_ID,Age,Gender,Surgical_Findings,Surgical Cure,SPECT/CT,Ultrasound,4D CT Scan,Sestamibi,MRI,Ultrasound_Concordance,SPECT/CT_Concordance,4D CT Scan_Concordance,Sestamibi_Concordance,MRI_Concordance,Surgical Side,SPECT/CT-Sestamibi
0,2012-04-24,156895,77,F,Right Inferior,Yes,NaN,NaN,NaN,Right Inferior,Right Inferior,No Scan,No Scan,No Scan,Yes (Exact),Yes (Exact),Right,Sestamibi: Right Inferior
1,2012-05-08,182059,70,F,Left Inferior,Yes,NaN,Non-localizing,NaN,Non-localizing,Right Inferior,No,No Scan,No Scan,No,No,Left,Non-localizing
2,2012-05-22,150593,65,F,Left Inferior,Yes,NaN,Left Inferior,NaN,Left Inferior,NaN,Yes (Exact),No Scan,No Scan,Yes (Exact),No Scan,Left,Sestamibi: Left Inferior
3,2012-07-03,183346,69,F,"Left Superior, Right Superior",Yes,NaN,NaN,NaN,Left Inferior,Non-localizing,No Scan,No Scan,No Scan,Yes (Side),No,Left,Sestamibi: Left Inferior
4,2012-07-18,183255,74,F,Left Superior,Yes,NaN,Left,NaN,Left,NaN,Yes (Side),No Scan,No Scan,Yes (Side),No Scan,Left,Sestamibi: Left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,2025-04-29,310063,48,M,"Left Superior, Right Superior",Yes,"Left, Right","Left Inferior, Right Inferior",NaN,NaN,NaN,Yes (Side),Yes (Side),No Scan,No Scan,No Scan,Left,"SPECT/CT: Left, Right"
401,2025-04-29,311569,47,F,Right Inferior,Yes,Right Inferior,NaN,Right Inferior,NaN,NaN,No Scan,Yes (Exact),Yes (Exact),No Scan,No Scan,Right,SPECT/CT: Right Inferior
402,2025-05-06,309738,73,F,Right Inferior,Yes,Right Inferior,Non-localizing,"Left Superior, Right Inferior",NaN,NaN,No,Yes (Exact),No,No Scan,No Scan,Right,SPECT/CT: Right Inferior
403,2025-05-13,194173,73,F,"Left Inferior, Right Superior, Right Inferior",Yes,Right Superior,Non-localizing,Right,NaN,NaN,No,No,No,No Scan,No Scan,Left,SPECT/CT: Right Superior


In [6]:
# Define modality
modality = 'SPECT/CT-Sestamibi'
imaging_side_col = 'SPECT/CT-Sestamibi Imaging Side'

# Apply extract_side function
preop_imaging_df[imaging_side_col] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized (Left or Right)
localized_spectmibi = preop_imaging_df[preop_imaging_df[imaging_side_col].isin(['Left', 'Right'])].copy()
total_localized_spectmibi = localized_spectmibi.shape[0]


In [9]:
# Extract surgical side if not already done
if 'Surgical Side' not in preop_imaging_df.columns:
    preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

# Apply generalized check_concordance function
localized_spectmibi['Calculated Concordance'] = localized_spectmibi.apply(
    lambda row: check_concordance(row, modality), axis=1
)

,Procedure_Date,Patient_ID,Age,Gender,Surgical_Findings,Surgical Cure,SPECT/CT,Ultrasound,4D CT Scan,Sestamibi,MRI,Ultrasound_Concordance,SPECT/CT_Concordance,4D CT Scan_Concordance,Sestamibi_Concordance,MRI_Concordance,Surgical Side,SPECT/CT-Sestamibi,SPECT/CT-Sestamibi Imaging Side
0,2012-04-24,156895,77,F,Right Inferior,Yes,NaN,NaN,NaN,Right Inferior,Right Inferior,No Scan,No Scan,No Scan,Yes (Exact),Yes (Exact),Right,Sestamibi: Right Inferior,Right
1,2012-05-08,182059,70,F,Left Inferior,Yes,NaN,Non-localizing,NaN,Non-localizing,Right Inferior,No,No Scan,No Scan,No,No,Left,Non-localizing,Unknown
2,2012-05-22,150593,65,F,Left Inferior,Yes,NaN,Left Inferior,NaN,Left Inferior,NaN,Yes (Exact),No Scan,No Scan,Yes (Exact),No Scan,Left,Sestamibi: Left Inferior,Left
3,2012-07-03,183346,69,F,"Left Superior, Right Superior",Yes,NaN,NaN,NaN,Left Inferior,Non-localizing,No Scan,No Scan,No Scan,Yes (Side),No,Left,Sestamibi: Left Inferior,Left
4,2012-07-18,183255,74,F,Left Superior,Yes,NaN,Left,NaN,Left,NaN,Yes (Side),No Scan,No Scan,Yes (Side),No Scan,Left,Sestamibi: Left,Left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,2025-04-29,310063,48,M,"Left Superior, Right Superior",Yes,"Left, Right","Left Inferior, Right Inferior",NaN,NaN,NaN,Yes (Side),Yes (Side),No Scan,No Scan,No Scan,Left,"SPECT/CT: Left, Right",Left
401,2025-04-29,311569,47,F,Right Inferior,Yes,Right Inferior,NaN,Right Inferior,NaN,NaN,No Scan,Yes (Exact),Yes (Exact),No Scan,No Scan,Right,SPECT/CT: Right Inferior,Right
402,2025-05-06,309738,73,F,Right Inferior,Yes,Right Inferior,Non-localizing,"Left Superior, Right Inferior",NaN,NaN,No,Yes (Exact),No,No Scan,No Scan,Right,SPECT/CT: Right Inferior,Right
403,2025-05-13,194173,73,F,"Left Inferior, Right Superior, Right Inferior",Yes,Right Superior,Non-localizing,Right,NaN,NaN,No,No,No,No Scan,No Scan,Left,SPECT/CT: Right Superior,Right


In [8]:
# Count total concordant (either exact or side match)
total_concordant = localized_spectmibi['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()

# Calculate percentage
side_concordance_percentage_spectmibi = (total_concordant / total_localized_spectmibi * 100) if total_localized_spectmibi > 0 else None

# Create results dataframe for clean display
side_concordance_spectmibi_df = pd.DataFrame({
    'Metric': [
        'Total SPECT/CT-Sestamibi scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_spectmibi,
        f"{total_concordant} ({side_concordance_percentage_spectmibi:.2f}%)"
    ]
})

# Display results
display(side_concordance_spectmibi_df)

,Metric,Value
0,Total SPECT/CT-Sestamibi scans with localization,285
1,Scans matching surgical side or exact gland,270 (94.74%)


---

### Interpretation

Positive SPECT/CT + Sestamibi scans showed **high concordance (95%) with surgical findings**, indicating strong diagnostic utility when both modalities localize.



### Next Steps

- Compare these results to **4D CT and combined modality performance** to guide imaging pathway recommendations in the manuscript.